In [1]:
import sqlite3
import pandas as pd
import re
import nltk
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
np.set_printoptions(threshold=np.inf)

# In this notebook we only read, clean and save data as csv. 
We don't tokenise it because with tokenised column data will take more than 100mb and we will not be able to save it to git.

Also because there are multiple ways to tokenise data.

So it's up to every separate model training file to tokenise the data.

## 1. Read data

In [15]:
# Connect to the database
DB_FILE = '/local/DSPT/data/nlp-data.db'
con = sqlite3.connect(DB_FILE)
cur = con.cursor()

# Define the SQL query
QUERY = """
    SELECT 
        Documents.DocumentID, 
        RawTexts.Text, 
        Labels.NumericValue AS Label, 
        Labels.Type AS LabelType, 
        Labels.StringValue AS LabelName,  
        Documents.Type AS DocumentType,
        RawTexts.LengthCharacters,
        RawTexts.HasEmoji
    FROM Documents
    INNER JOIN RawTexts ON Documents.RawTextID = RawTexts.RawTextID
    INNER JOIN Labels ON Documents.LabelID = Labels.LabelID;
"""

# Execute the query and load data into a DataFrame
reviews = pd.read_sql_query(QUERY, con)
con.close()

## 2. Clean data

In [16]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'_', '', text)
    return text

# Clean the text data
reviews['cleaned_text'] = reviews['Text'].apply(clean_text)
reviews = reviews[reviews['cleaned_text'] != '']

stop_words = set(stopwords.words('english'))

# Drow rows that have None in cleaned_text column
reviews = reviews.dropna(subset=['cleaned_text'])

### 3. Map amazon rating to -1/0/1

In [17]:
def map_amazon_rating(rating):
    if rating <= 2:
        return -1
    elif rating == 3:
        return 0
    else:
        return 1

reviews['LabelMapped'] = reviews.apply(lambda row: map_amazon_rating(row["Label"]) if row['LabelType'] == 'StarRating' else row["Label"], axis=1)

In [18]:
reviews['LabelMapped'].unique()

array([-1,  0,  1])

### 4. Add emoticons as columns

In [20]:
reviews['emoticon1'] = reviews['Text'].str.contains(":-\)").astype(int)
reviews['emoticon2'] = reviews['Text'].str.contains(':-/').astype(int)
reviews['emoticon3'] = reviews['Text'].str.contains(':-\(').astype(int)

### 5. Extract hashtags

### 5. Save to csv

In [21]:
reviews.to_csv('data/prepared_data.csv', index=False)

In [3]:
reviews = pd.read_csv("data/prepared_data.csv")
reviews = reviews[:100]

In [5]:
# Split the 'text' column by space and return as list
reviews['text_split'] = reviews['cleaned_text'].str.split()


In [6]:
# Combine all lists in the 'text_split' column into one list
combined_list = sum(reviews['text_split'], [])

print("\nCombined List:")
print(combined_list)


Combined List:
['arrived', 'broken', 'manufacturer', 'defect', 'two', 'of', 'the', 'legs', 'of', 'the', 'base', 'were', 'not', 'completely', 'formed', 'so', 'there', 'was', 'no', 'way', 'to', 'insert', 'the', 'casters', 'i', 'unpackaged', 'the', 'entire', 'chair', 'and', 'hardware', 'before', 'noticing', 'this', 'so', 'ill', 'spend', 'twice', 'the', 'amount', 'of', 'time', 'boxing', 'up', 'the', 'whole', 'useless', 'thing', 'and', 'send', 'it', 'back', 'with', 'a', 'star', 'review', 'of', 'part', 'of', 'a', 'chair', 'i', 'never', 'got', 'to', 'sit', 'in', 'i', 'will', 'go', 'so', 'far', 'as', 'to', 'include', 'a', 'picture', 'of', 'what', 'their', 'injection', 'molding', 'and', 'quality', 'assurance', 'process', 'missed', 'though', 'i', 'will', 'be', 'hesitant', 'to', 'buy', 'again', 'it', 'makes', 'me', 'wonder', 'if', 'there', 'arent', 'missing', 'structures', 'and', 'supports', 'that', 'dont', 'impede', 'the', 'assembly', 'process', 'the', 'cabinet', 'dot', 'were', 'all', 'detached

In [7]:
# Count the frequencies of words
word_frequencies = Counter(combined_list)

print("\nWord Frequencies:")
print(word_frequencies)


Word Frequencies:
Counter({'the': 177, 'i': 101, 'it': 101, 'and': 91, 'a': 79, 'to': 68, 'of': 49, 'was': 44, 'this': 44, 'not': 42, 'in': 36, 'is': 34, 'my': 33, 'for': 33, 'but': 30, 'with': 27, 'on': 26, 'that': 23, 'like': 23, 'have': 22, 'out': 21, 'product': 20, 'very': 20, 'as': 18, 'after': 17, 'so': 15, 'one': 15, 'its': 15, 'get': 15, 'be': 14, 'work': 14, 'had': 14, 'we': 13, 'does': 13, 'all': 12, 'even': 12, 'used': 12, 'no': 11, 'would': 11, 'are': 11, 'never': 10, 'if': 10, 'from': 10, 'than': 10, 'they': 10, 'you': 10, 'disappointed': 10, 'money': 10, 'few': 10, 'at': 10, 'then': 9, 'when': 9, 'bought': 9, 'because': 9, 'screen': 9, 'or': 9, 'just': 9, 'got': 8, 'will': 8, 'received': 8, 'your': 8, 'these': 8, 'been': 8, 'waste': 8, 'other': 8, 'well': 8, 'terrible': 8, 'two': 7, 'time': 7, 'dont': 7, 'off': 7, 'day': 7, 'top': 7, 'them': 7, 'return': 7, 'now': 7, 'cannot': 7, 'about': 7, 'an': 7, 'any': 7, 'box': 7, 'hair': 7, 'up': 6, 'back': 6, 'quality': 6, 'order

In [9]:
print("Words ordered by frequency:")
ordered_words = word_frequencies.most_common()
for word, freq in ordered_words:
    print(f"{word}: {freq}")

Words ordered by frequency:
the: 177
i: 101
it: 101
and: 91
a: 79
to: 68
of: 49
was: 44
this: 44
not: 42
in: 36
is: 34
my: 33
for: 33
but: 30
with: 27
on: 26
that: 23
like: 23
have: 22
out: 21
product: 20
very: 20
as: 18
after: 17
so: 15
one: 15
its: 15
get: 15
be: 14
work: 14
had: 14
we: 13
does: 13
all: 12
even: 12
used: 12
no: 11
would: 11
are: 11
never: 10
if: 10
from: 10
than: 10
they: 10
you: 10
disappointed: 10
money: 10
few: 10
at: 10
then: 9
when: 9
bought: 9
because: 9
screen: 9
or: 9
just: 9
got: 8
will: 8
received: 8
your: 8
these: 8
been: 8
waste: 8
other: 8
well: 8
terrible: 8
two: 7
time: 7
dont: 7
off: 7
day: 7
top: 7
them: 7
return: 7
now: 7
cannot: 7
about: 7
an: 7
any: 7
box: 7
hair: 7
up: 6
back: 6
quality: 6
ordered: 6
more: 6
doesnt: 6
by: 6
only: 6
use: 6
see: 6
came: 6
didnt: 6
im: 6
has: 6
jewelry: 6
mattress: 6
could: 6
broken: 5
there: 5
thing: 5
though: 5
me: 5
first: 5
order: 5
do: 5
try: 5
better: 5
month: 5
item: 5
hard: 5
purchase: 5
color: 5
size: 5
gre